See https://github.com/facebook/Ax/issues/743

In [1]:
%pip install ax-platform

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.7/730.7 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.8 MB/s eta 0:00:00


In [22]:
# %% imports
import numpy as np
import pandas as pd

from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler, normalize

from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep
from ax.modelbridge.registry import Models

from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties

Create `X_train` and `y_train` using sklearn's `make_regression` function. We will use
this data to train a model. You can load your data in via CSV files instead.

In [23]:
n_train = 7
unique_components = ["filler_A", "filler_B", "resin_A", "resin_B", "resin_C"]
objective_names = ["yield_strength", "elongation"]

X_train, y_train = make_regression(
    n_samples=n_train,
    n_features=5,
    n_informative=5,
    n_targets=2,
    noise=0.1,
    random_state=10,
)

X_train = MinMaxScaler().fit_transform(X_train)
X_train = normalize(X_train, norm="l1")

y_train[:, 0] = (
    MinMaxScaler(feature_range=(0, 100))
    .fit_transform(y_train[:, 0].reshape(-1, 1))
    .ravel()
)
y_train[:, 1] = (
    MinMaxScaler(feature_range=(0, 5))
    .fit_transform(y_train[:, 1].reshape(-1, 1))
    .ravel()
)

X_train = pd.DataFrame(X_train, columns=unique_components)
y_train = pd.DataFrame(y_train, columns=objective_names)

In [24]:
X_train

,filler_A,filler_B,resin_A,resin_B,resin_C
0,0.263170,0.161356,0.319319,0.177511,0.078643
1,0.233281,0.191723,0.165822,0.409174,0.000000
2,0.067263,0.225302,0.341151,0.186902,0.179382
3,0.242482,0.000000,0.000000,0.363517,0.394001
4,0.000000,0.346443,0.318884,0.272197,0.062476
5,0.293613,0.293613,0.183609,0.208137,0.021027
6,0.268568,0.312212,0.096869,0.000000,0.322350


In [25]:
y_train

,yield_strength,elongation
0,11.076321,1.926677
1,31.772108,1.780739
2,75.022848,2.872787
3,43.808779,0.000000
4,44.688723,2.628655
5,100.000000,5.000000
6,0.000000,2.048223


In [50]:
# Ax-specific
filler_upper_bound = 0.825
resin_upper_bound = 1 - 0.575
loose_parameters = [
    {"name": component, "type": "range", "bounds": [0.0, 1.0]}
    for component in unique_components
]

tight_parameters = [
    {
        "name": component,
        "type": "range",
        "bounds": [
            0.0,
            filler_upper_bound if "filler" in component else resin_upper_bound,
        ],
    }
    for component in unique_components[:-1]
]

In [51]:
loose_parameters

[{'name': 'filler_A', 'type': 'range', 'bounds': [0.0, 1.0]},
 {'name': 'filler_B', 'type': 'range', 'bounds': [0.0, 1.0]},
 {'name': 'resin_A', 'type': 'range', 'bounds': [0.0, 1.0]},
 {'name': 'resin_B', 'type': 'range', 'bounds': [0.0, 1.0]},
 {'name': 'resin_C', 'type': 'range', 'bounds': [0.0, 1.0]}]

In [52]:
tight_parameters

[{'name': 'filler_A', 'type': 'range', 'bounds': [0.0, 0.825]},
 {'name': 'filler_B', 'type': 'range', 'bounds': [0.0, 0.825]},
 {'name': 'resin_A', 'type': 'range', 'bounds': [0.0, 0.42500000000000004]},
 {'name': 'resin_B', 'type': 'range', 'bounds': [0.0, 0.42500000000000004]}]

In [53]:
separator = " + "
# composition_constraint = separator.join(unique_components[:-1]) + " <= 1.0"
filler_components = [
    component for component in unique_components[:-1] if "filler" in component
]
# resin_components = [
#     component for component in unique_components[:-1] if "resin" in component
# ]
filler_lower_constraint = (
    separator.join(filler_components) + f" >= {1 - resin_upper_bound}"
)
filler_upper_constraint = (
    separator.join(filler_components) + f" <= {filler_upper_bound}"
)
parameter_constraints = [
    # composition_constraint,
    filler_lower_constraint,
    filler_upper_constraint,
]
parameter_constraints

['filler_A + filler_B >= 0.575', 'filler_A + filler_B <= 0.825']

In [63]:
# skip the pseudo-random suggested points by specifying a custom generation strategy
import torch

num_components = len(unique_components)

inequality_constraints = [
    (
        torch.arange(num_components, dtype=torch.long),
        torch.tensor(
            [1 if "filler" in component else 0 for component in unique_components],
            dtype=torch.double,
        ),
        filler_upper_bound,
    ),
    (
        torch.arange(num_components, dtype=torch.long),
        torch.tensor(
            [-1 if "filler" in component else 0 for component in unique_components],
            dtype=torch.double,
        ),
        1 - resin_upper_bound,
    )
]

# sum(x) == 1
equality_constraints = [
    (
        torch.arange(num_components, dtype=torch.long),
        torch.ones(num_components, dtype=torch.double),
        1,
    )
]

gs = GenerationStrategy(
    steps=[
        # 2. Bayesian optimization step (requires data obtained from previous phase and learns
        # from all data available at the time of each new candidate generation call)
        GenerationStep(
            # model=Models.FULLYBAYESIANMOO,
            model=Models.MOO,
            model_kwargs={"fit_out_of_design": True},
            model_gen_kwargs={
                "model_gen_options": {
                    "optimizer_kwargs": {
                        "inequality_constraints": inequality_constraints,
                        "equality_constraints": equality_constraints,
                    },
                }
            },
            num_trials=-1,  # No limitation on how many trials should be produced from this step
            max_parallelism=None,  # Parallelism limit for this step, often lower than for Sobol
            # More on parallelism vs. required samples in BayesOpt:
            # https://ax.dev/docs/bayesopt.html#tradeoff-between-parallelism-and-total-number-of-trials
        ),
    ]
)

objectives = {
    objective_name: ObjectiveProperties(minimize=False)
    for objective_name in objective_names
}

# setup the experiment
experiment_name = "dummy"
ax_client = AxClient(generation_strategy=gs)
ax_client.create_experiment(
    name=experiment_name,
    parameters=loose_parameters,
    objectives=objectives,
    immutable_search_space_and_opt_config=False,
)

ax_client_tmp = AxClient(generation_strategy=gs)
ax_client_tmp.create_experiment(
    name=experiment_name,
    parameters=tight_parameters,
    objectives=objectives,
    immutable_search_space_and_opt_config=False,
)

[INFO 02-18 13:54:38] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-18 13:54:38] ax.service.utils.instantiation: Due to non-specification, we will use the heuristic for selecting objective thresholds.
[INFO 02-18 13:54:38] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter filler_A. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 02-18 13:54:38] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter filler_B. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 02-18 13:54:38] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter resin_A. If that i

In [64]:
# attach the training data
for i in range(n_train):
    ax_client.attach_trial(X_train.iloc[i].to_dict())
    ax_client.complete_trial(trial_index=i, raw_data=y_train.iloc[i, :].to_dict())

[INFO 02-18 13:54:38] ax.service.ax_client: Attached custom parameterization {'filler_A': 0.26317, 'filler_B': 0.161356, 'resin_A': 0.319319, 'resin_B': 0.177511, 'resin_C': 0.078643} as trial 0.
[INFO 02-18 13:54:38] ax.service.ax_client: Completed trial 0 with data: {'yield_strength': (11.076321, None), 'elongation': (1.926677, None)}.
[INFO 02-18 13:54:38] ax.service.ax_client: Attached custom parameterization {'filler_A': 0.233281, 'filler_B': 0.191723, 'resin_A': 0.165822, 'resin_B': 0.409174, 'resin_C': 0.0} as trial 1.
[INFO 02-18 13:54:38] ax.service.ax_client: Completed trial 1 with data: {'yield_strength': (31.772108, None), 'elongation': (1.780739, None)}.
[INFO 02-18 13:54:38] ax.service.ax_client: Attached custom parameterization {'filler_A': 0.067263, 'filler_B': 0.225302, 'resin_A': 0.341151, 'resin_B': 0.186902, 'resin_C': 0.179382} as trial 2.
[INFO 02-18 13:54:38] ax.service.ax_client: Completed trial 2 with data: {'yield_strength': (75.022848, None), 'elongation': (2

Produce a *batch* of five next suggested experiments. **Be sure to only run this once.**

In [65]:
next_experiments, optimization_complete = ax_client.get_next_trials(max_trials=5)

TypeError: ax.models.torch.botorch_defaults.scipy_optimizer() got multiple values for keyword argument 'inequality_constraints'

Warmup:   3%|▎         | 21/768 [00:20, 11.86it/s, step size=1.18e-01, acc. prob=0.751]

In [73]:
from pprint import pprint
print("next suggested experiments:")
pprint(next_experiments)

next suggested experiments:
{7: {'filler_A': 0.3260161948842118,
     'filler_B': 0.34277081285749944,
     'resin_A': 0.16871984075941424,
     'resin_B': 0.13128015924076872},
 8: {'filler_A': 0.33265439569579575,
     'filler_B': 0.3673456043040991,
     'resin_A': 0.16762655987294398,
     'resin_B': 0.13237344012693003},
 9: {'filler_A': 0.412904324680518,
     'filler_B': 0.28709567531948066,
     'resin_A': 0.0894530622435886,
     'resin_B': 0.21054693775640562},
 10: {'filler_A': 1.665008747311923e-13,
      'filler_B': 0.7,
      'resin_A': 0.16631813258167671,
      'resin_B': 0.13368186741806554},
 11: {'filler_A': 4.0860711084152173e-16,
      'filler_B': 0.6999999999999957,
      'resin_A': 0.05014880409698304,
      'resin_B': 0.21968920802447037}}


In [74]:
pareto_optimal_parameters = ax_client.get_pareto_optimal_parameters()

Sample: 100%|██████████| 768/768 [00:53, 14.33it/s, step size=2.70e-01, acc. prob=0.901]
[INFO 02-18 11:49:31] ax.service.utils.best_point: Using inferred objective thresholds: [ObjectiveThreshold(elongation >= 2.032803509426325), ObjectiveThreshold(yield_strength >= -3.770957883207984)], as objective thresholds were not specified as part of the optimization configuration on the experiment.


In [75]:
pprint(pareto_optimal_parameters)

{}
